In [1]:
import sqlite3
import pandas as pd

In [2]:
db = sqlite3.connect('issues.db')
cur = db.cursor()

In [3]:
T = '2020-10-10'

1 вариант

In [4]:
col1 = pd.Series([], dtype='object')
col2 = pd.Series([], dtype='object')
col3 = pd.Series([], dtype='object')

In [5]:
cur.execute("""select issue_key,
                    min(datetime(status_begin/1000, 'unixepoch')) as start_time,
                    status
             from history
             where date(:T) between datetime(status_begin/1000, 'unixepoch') and datetime(status_end/1000, 'unixepoch') and
                   status not in ('Closed', 'Resolved')
             group by issue_key""", (T,))
for row in cur.fetchall():
    col1 = pd.concat([col1, pd.Series(row[0])])
    col2 = pd.concat([col2, pd.Series(row[1])])
    col3 = pd.concat([col3, pd.Series(row[2])])
#     col1 = col1.append(pd.Series(row[0]))
#     col2 = col2.append(pd.Series(row[1]))
#     col3 = col3.append(pd.Series(row[2])) появляется future warning

In [6]:
df = pd.DataFrame({'issue_key':col1.reset_index(drop=True),
             'start_time':col2.reset_index(drop=True),
             'status':col3.reset_index(drop=True)})

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   issue_key   29 non-null     object
 1   start_time  29 non-null     object
 2   status      29 non-null     object
dtypes: object(3)
memory usage: 824.0+ bytes


In [8]:
df['start_time'] = pd.to_datetime(df['start_time']).dt.tz_localize(None)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   issue_key   29 non-null     object        
 1   start_time  29 non-null     datetime64[ns]
 2   status      29 non-null     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 824.0+ bytes


In [10]:
df

,issue_key,start_time,status
0,A-20577,2019-11-05 09:33:48,Open
1,A-25044,2020-07-13 13:44:24,Open
2,A-26168,2020-09-24 16:47:05,In Review
3,A-26389,2020-09-23 09:35:04,Open
4,A-26485,2020-10-07 13:27:25,In Owl
5,A-26500,2020-10-07 13:27:25,Waiting for integration
6,A-26604,2020-10-01 13:15:06,Open
7,A-26627,2020-10-02 11:00:14,In Progress
8,A-26681,2020-10-07 13:46:48,Waiting for integration
9,A-26683,2020-10-06 11:29:04,Blocked


2 вариант

In [11]:
cur.execute("""select issue_key,
                    min(datetime(status_begin/1000, 'unixepoch')) as start_time,
                    status
             from history
             where date(:T) between datetime(status_begin/1000, 'unixepoch') and datetime(status_end/1000, 'unixepoch') and
                   status not in ('Closed', 'Resolved')
             group by issue_key""", (T,))
cur.fetchall()

[('A-20577', '2019-11-05 09:33:48', 'Open'),
 ('A-25044', '2020-07-13 13:44:24', 'Open'),
 ('A-26168', '2020-09-24 16:47:05', 'In Review'),
 ('A-26389', '2020-09-23 09:35:04', 'Open'),
 ('A-26485', '2020-10-07 13:27:25', 'In Owl'),
 ('A-26500', '2020-10-07 13:27:25', 'Waiting for integration'),
 ('A-26604', '2020-10-01 13:15:06', 'Open'),
 ('A-26627', '2020-10-02 11:00:14', 'In Progress'),
 ('A-26681', '2020-10-07 13:46:48', 'Waiting for integration'),
 ('A-26683', '2020-10-06 11:29:04', 'Blocked'),
 ('A-26711', '2020-10-09 09:59:17', 'In Testing'),
 ('A-26717', '2020-10-08 13:34:52', 'Waiting for integration'),
 ('A-26783', '2020-10-08 14:15:15', 'In Testing'),
 ('A-26809', '2020-10-09 15:01:50', 'Waiting for integration'),
 ('A-26814', '2020-10-09 11:25:44', 'In Progress'),
 ('A-26845', '2020-10-09 10:29:42', 'Open'),
 ('C-6951', '2020-10-08 12:25:46', 'In Progress'),
 ('D-1555', '2020-10-01 09:26:43', 'Blocked'),
 ('D-1869', '2020-06-15 08:35:19', 'In Progress'),
 ('D-1912', '2020-1